In [24]:
from utils import *

hete = Heterogeneity()
hete.check_torch_gpu()

-------------------------------------------------
------------------ VERSION INFO -----------------
Conda Environment: torchy | Python version: 3.8.16 (default, Mar  2 2023, 03:18:16) [MSC v.1916 64 bit (AMD64)]
Torch version: 2.0.1
Torch build with CUDA? True
# Device(s) available: 1, Name(s): Quadro P520



In [8]:
hete_facies = hete.load_facies()
hete_fluv, hete_gaus = hete.load_perm_poro()

Facies shape: (1000, 256, 256)
Permf: (1000, 65536) | Permg: (1000, 65536)
Fluvial heterogeneity: (1000, 256, 256, 2) | Gaussian heterogeneity: (1000, 256, 256, 2)


***
# END